In [1]:
import hickle as hkl

In [2]:
import numpy as np
np.random.seed(1234)

In [3]:
PESeq = hkl.load('./Gen_data/11_Gen_Pro_Ehr.hkl')

## pre-process data set

In [4]:
X=[]
Y=[]
for index in range(len(PESeq['label'])):
    Y.append(PESeq['label'][index])
    X.append(np.append(PESeq['Pro-Kmer'][index],PESeq['Ehr-Kmer'][index],axis=1))
X = np.array(X)
Y  = np.array(Y)

In [5]:
VALIDATION_SPLIT =0.1

index = range(len(PESeq['label']))
np.random.shuffle(index)
nb_validation_samples = int(VALIDATION_SPLIT*len(Y))

In [6]:
x_train = X[index[: - nb_validation_samples]]
y_train = Y[index[:-nb_validation_samples]]
x_val = X[index[-nb_validation_samples:]]
y_val =Y[index[-nb_validation_samples:]]

In [7]:
from keras.utils.np_utils import to_categorical

Using Theano backend.
Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5105)


In [8]:
y_train = to_categorical(y_train) #one-hot 编码labels
y_val = to_categorical(np.asarray(y_val))

# Make a Network

In [9]:
from keras.layers import Embedding
from keras.layers import Dense,Input,Activation
from keras.layers import Embedding, LSTM, Bidirectional,GRU,InputLayer
from keras.models import Model,Sequential
from  keras.regularizers import ActivityRegularizer
from keras.layers.core import Dropout
from keras import layers

import os
os.environ['KERAS_BACKEND']='theano'
from keras.engine.topology import Layer
from keras import initializations
from keras import backend as K

class AttLayer(Layer):
    def __init__(self, **kwargs):
        self.init = initializations.get('normal')
        #self.input_spec = [InputSpec(ndim=3)]
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape)==3
        #self.W = self.init((input_shape[-1],1))
        self.W = self.init((input_shape[-1],))
        #self.input_spec = [InputSpec(shape=input_shape)]
        self.trainable_weights = [self.W]
        super(AttLayer, self).build(input_shape)  # be sure you call this somewhere!

    def call(self, x, mask=None):


        M = K.tanh(x)
        alpha = K.dot(M,self.W)#.dimshuffle(0,2,1)

        ai = K.exp(alpha)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')
        weighted_input = x*weights.dimshuffle(0,1,'x')
        return K.tanh(weighted_input.sum(axis=1))
        '''
        eij = K.tanh(K.dot(x, self.W))

        ai = K.exp(eij)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')

        weighted_input = x*weights.dimshuffle(0,1,'x')
        return weighted_input.sum(axis=1)
        '''
    def get_output_shape_for(self, input_shape):
        return (input_shape[0], input_shape[-1])


In [21]:
kmer_input = Input(shape=(4096,2), dtype='float32')
l_lstm =Bidirectional(LSTM(20,return_sequences=True))(kmer_input)
l_lstm_drop = Dropout(0.3)(l_lstm)
#l_att = AttLayer()(l_lstm_drop)
#l_att_drop = Dropout(0.5)(l_att)
l_lstm_2 = Bidirectional(LSTM(20,return_sequences=False))(l_lstm_drop)
l_lstm_2_drop = Dropout(0.3)(l_lstm_2)
preds = Dense(len( y_train[0]), activation='softmax',activity_regularizer= ActivityRegularizer(l2=0.005))(l_lstm_2_drop)
model  = Model (kmer_input,preds)

In [22]:
model.compile(loss='mse',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - attention LSTM network")
model.summary()

model fitting - attention LSTM network
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_4 (InputLayer)             (None, 4096, 2)       0                                            
____________________________________________________________________________________________________
bidirectional_5 (Bidirectional)  (None, 4096, 40)      3680        input_4[0][0]                    
____________________________________________________________________________________________________
dropout_5 (Dropout)              (None, 4096, 40)      0           bidirectional_5[0][0]            
____________________________________________________________________________________________________
bidirectional_6 (Bidirectional)  (None, 40)            9760        dropout_5[0][0]                  
____________________________________________________

In [ ]:
model.fit(x_train, y_train, validation_data=(x_val, y_val), nb_epoch=20, batch_size=100)

Train on 21377 samples, validate on 2375 samples
Epoch 1/20
21377/21377 [==============================] - 6612s - loss: 0.4991 - acc: 0.5428 - val_loss: 0.4959 - val_acc: 0.5364
Epoch 2/20
21377/21377 [==============================] - 6572s - loss: 0.4975 - acc: 0.5489 - val_loss: 0.4946 - val_acc: 0.5554
Epoch 3/20
21377/21377 [==============================] - 8267s - loss: 0.4967 - acc: 0.5521 - val_loss: 0.4951 - val_acc: 0.5436
Epoch 4/20
21377/21377 [==============================] - 9686s - loss: 0.4966 - acc: 0.5537 - val_loss: 0.4944 - val_acc: 0.5608
Epoch 5/20
21377/21377 [==============================] - 9682s - loss: 0.4964 - acc: 0.5574 - val_loss: 0.4942 - val_acc: 0.5608
Epoch 6/20
21377/21377 [==============================] - 9683s - loss: 0.4963 - acc: 0.5576 - val_loss: 0.4941 - val_acc: 0.5587
Epoch 7/20
21377/21377 [==============================] - 9649s - loss: 0.4960 - acc: 0.5598 - val_loss: 0.4940 - val_acc: 0.5646
Epoch 8/20
21377/21377 [=================

KeyboardInterrupt: 

In [ ]:
model.fit(x_train, y_train, validation_data=(x_val, y_val), nb_epoch=20, batch_size=100)

Train on 21377 samples, validate on 2375 samples
Epoch 1/20


## 单层LSTM